In [75]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np
from sojo.pstabilizer import group_instructorss_by_qubits, Instructor
from sojo.pstabilizer import weightss_to_lambda, instructor_to_lut
from sojo.utils import word_to_index, char_to_index, index_to_word
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [83]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.clustering()

In [78]:
(ins.xclusters)

[[[0, 1], [1, 2], [2, 0]],
 [[0, 1], [1, 2], [2, 0]],
 [[0, 1], [1, 2], [2, 0]],
 [[0, 1], [1, 2], [2, 0]],
 [[0, 1], [1, 2], [2, 0]]]

In [84]:
LUT = instructor_to_lut(ins)

(5, 3, 3, 4)


In [92]:
num_qubits = 3
term = ['zii', 'ixx']
k = 0
# Init
# 2**n is th

lambdas = [1, 1]
indices = [4, 3]
weightss = []
for index in indices:
    weights = []
    word = index_to_word(index, num_qubits)
    for j, char in enumerate(word):
        i_in_lut = char_to_index(char) - 1
        if i_in_lut == -1:
            weights.append([1,0,0,0])
        else: 
            weights.append(LUT[k][j][i_in_lut])
    weightss.append(weights)
weightss = np.array(weightss)
np.save('./test_weight_to_lambdas/weight_1.npy', weightss)
np.save('./test_weight_to_lambdas/lambdas_1.npy', lambdas)
lambdas = weightss_to_lambda(weightss, lambdas)
np.save('./test_weight_to_lambdas/output_1.npy', lambdas)
print(lambdas)
indices = np.nonzero(lambdas)[0]
lambdas = np.array(lambdas)[indices]

[ 0.          0.62021043  0.09762136 -0.77833739  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.82559634  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.06766809  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.56018899  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]


In [81]:
print(indices)
print(lambdas)

[21 22 23 25 26 27 29 30 31 37 38 39 41 42 43 45 46 47 53 54 55 57 58 59
 61 62 63]
[ 0.15878178 -1.10006246 -0.35056599  0.04976422  1.13451404  0.67093598
 -0.04101927  0.58196531  0.24773958 -0.09730854 -0.32850662 -0.31439582
 -0.1288524   0.4929354   0.073476    0.00533981  0.20481789  0.11584584
  0.40673739 -0.03582383  0.5704583   0.40038045 -0.39074691  0.37390906
 -0.05014041 -0.06714194 -0.10809335]


In [82]:
import polars as pl

def map_cx(indices: np.ndarray, lambdas: np.ndarray, cxs: np.ndarray, num_qubits: int):
	"""Mapping multple CX gates on a given indices and lambdas

	Args:
		indices (np.ndarray): Words after converting to integers
		lambdas (np.ndarray): Corresponding lambdas
		cxs (np.ndarray): List of [control, target] pairs
		num_qubits (int)

	Returns:
		indices, lambdas
	"""
	for control, target in cxs:
		df = pl.read_csv(f'./sojo/db/{num_qubits}_{control}_{target}_cx.csv')
		selected_rows = df[indices]['out'].to_numpy()
		indices = np.abs(selected_rows)
		lambdas[np.where(selected_rows < 0)[0]] *= -1
	return indices, lambdas

map_cx(np.array([3, 1, 27, 5]), np.array([1,2,3,4]), [[0, 2], [1, 0]], 3)

(array([63,  1, 54, 21], dtype=int64), array([1, 2, 3, 4]))